In [6]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

In [7]:
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from io import StringIO
from statsmodels.tsa.stattools import adfuller

Проверить на стационарность ряд цены на нефть (https://finance.yahoo.com/quote/BZ%3DF/history?
period1=1185753600&period2=1666310400&interval=1d&filter=history
&frequency=1d&includeAdjustedClose=true).

In [10]:
ticker = "BZ=F"
start_date = "2007-06-30"
end_date = "2023-10-21"
brent_df = yf.download(ticker, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


- Проводим расширенный тест Дики-Фуллера.
 - Сформируем гипотезы:
    - Нулевая гипотеза (H0) предполагает наличие единичного корня - временной ряд нестационарен.
    - Альтернативная гипотеза (H1) предполагает отсутствие единичного корня - временной ряд стационарен.
- Статистика ADF рассчитывается на основе авторегрессионного уравнения. Если статистика меньше критического значения, нулевая гипотеза отвергается.
- В тесте будет рассчитано p-value. Установим уровень значимости 0.05 и будем отвергать нуль гипотезу при превышении p-value этого уровня.

In [11]:
adf_test = adfuller(brent_df ['Close'])
adf_test

(-2.213616022444665,
 0.20134079354023632,
 15,
 4011,
 {'1%': -3.4319813861701927,
  '5%': -2.862260857164274,
  '10%': -2.5671537198514356},
 15583.025623274)

Мы получили статистику -2.21, что меньше критического значения для 5% уровня значимости -2.86. p-value 0.201 > 0.05
Таким образом, не получилось отвергнуть нуль гипотезу, временной ряд нестационарен